<a href="https://colab.research.google.com/github/soujanyakd/Project_CNN_RNN/blob/main/RNN_UNSW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive. mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
pip install -q keras

In [58]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import seaborn as sns
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter


import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer


import pydot

In [59]:
#uploading training dataset
test_data= pd.read_csv("/content/drive/MyDrive/UNSW_NB15_training-set.csv",header=None)
print('-------Test data--------')
print(test_data[43].value_counts())
print(test_data.shape)
print('-------------------------')
test_data.drop(test_data.index[0],inplace = True)
test_data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


-------Test data--------
Normal            37000
Generic           18871
Exploits          11132
Fuzzers            6062
DoS                4089
Reconnaissance     3496
Analysis            677
Backdoor            583
Shellcode           378
Worms                44
attack_cat            1
Name: 43, dtype: int64
(82333, 45)
-------------------------


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
1,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,0,180363632,0,0,0,0.011,0,0,0,0,0,0,0,0,0,0,248,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
2,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,0,881000000,0,0,0,0.008,0,0,0,0,0,0,0,0,0,0,881,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
3,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,0,854400000,0,0,0,0.005,0,0,0,0,0,0,0,0,0,0,534,0,0,0,3,2,1,1,1,3,0,0,0,1,3,0,Normal,0
4,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,0,600000000,0,0,0,0.006,0,0,0,0,0,0,0,0,0,0,450,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0
5,5,0.00001,udp,-,INT,2,0,2126,0,100000.0025,254,0,850400000,0,0,0,0.01,0,0,0,0,0,0,0,0,0,0,1063,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,82328,5e-06,udp,-,INT,2,0,104,0,200000,254,0,8.32e+07,0,0,0,0.005,0,0,0,0,0,0,0,0,0,0,52,0,0,0,1,2,2,1,1,2,0,0,0,2,1,0,Normal,0
82329,82329,1.1061,tcp,-,FIN,20,8,18062,354,24.4101,254,252,124104,2242.11,7,1,55.8801,143.7,4798.13,190.981,255,1072535109,3284291478,255,0.173208,0.100191,0.073017,903,44,0,0,1,1,2,1,1,1,0,0,0,3,2,0,Normal,0
82330,82330,0,arp,-,INT,1,0,46,0,0,0,0,0,0,0,0,60000.7,0,0,0,0,0,0,0,0,0,0,46,0,0,0,1,2,1,1,1,1,0,0,0,1,1,1,Normal,0
82331,82331,0,arp,-,INT,1,0,46,0,0,0,0,0,0,0,0,60000.7,0,10.9545,0,0,0,0,0,0,0,0,46,0,0,0,1,2,1,1,1,1,0,0,0,1,1,1,Normal,0


In [60]:
test_data.drop(test_data[test_data[43]=='Generic'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Fuzzers'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='DoS'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Backdoor'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Shellcode'].index, inplace = True)
test_data.drop(test_data[test_data[43]=='Worms'].index, inplace = True)
print(test_data[43].value_counts())

Normal            37000
Exploits          11132
Reconnaissance     3496
Analysis            677
Name: 43, dtype: int64


In [61]:
#uploading training dataset
train_data= pd.read_csv("/content/drive/MyDrive/UNSW_NB15_testing-set.csv",header= None)
print('-------Train data--------')
print(train_data[43].value_counts())
print(len(train_data))
print('-------------------------')
train_data.drop(train_data.index[0],inplace = True)
train_data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


-------Train data--------
Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
attack_cat            1
Name: 43, dtype: int64
175342
-------------------------


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
1,1,0.121478,tcp,-,FIN,6,4,258,172,74.08749,252,254,14158.94238,8495.365234,0,0,24.2956,8.375,30.177547,11.830604,255,621772692,2202533631,255,0,0,0,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
2,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,8395.112305,503571.3125,2,17,49.915,15.432865,61.426934,1387.77833,255,1417884146,3077387971,255,0,0,0,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
3,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,1572.271851,60929.23047,1,6,231.875571,102.737203,17179.58686,11420.92623,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
4,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2740.178955,3358.62207,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0,0,0,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
5,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,8561.499023,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175337,175337,9e-06,udp,dns,INT,2,0,114,0,111111,254,0,5.06667e+07,0,0,0,0.009,0,0,0,0,0,0,0,0,0,0,57,0,0,0,24,2,24,24,13,24,0,0,0,24,24,0,Generic,1
175338,175338,0.505762,tcp,-,FIN,10,8,620,354,33.6126,254,252,8826.29,4903.49,2,1,54.4001,66.9806,3721.07,120.178,255,3518776216,3453092386,255,0.09944,0.036895,0.062545,62,44,0,0,1,1,1,1,1,2,0,0,0,1,1,0,Shellcode,1
175339,175339,9e-06,udp,dns,INT,2,0,114,0,111111,254,0,5.06667e+07,0,0,0,0.009,0,0,0,0,0,0,0,0,0,0,57,0,0,0,12,2,3,3,3,13,0,0,0,3,12,0,Generic,1
175340,175340,9e-06,udp,dns,INT,2,0,114,0,111111,254,0,5.06667e+07,0,0,0,0.009,0,0,0,0,0,0,0,0,0,0,57,0,0,0,30,2,30,30,14,30,0,0,0,30,30,0,Generic,1


In [62]:
train_data.drop(train_data[train_data[43]=='Generic'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Fuzzers'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='DoS'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Backdoor'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Shellcode'].index, inplace = True)
train_data.drop(train_data[train_data[43]=='Worms'].index, inplace = True)
print(train_data[43].value_counts())

Normal            56000
Exploits          33393
Reconnaissance    10491
Analysis           2000
Name: 43, dtype: int64


In [63]:
#preprocessing
#Evaluation of the training dataset
train_data.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
dtype: int64

In [64]:

train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101884 entries, 1 to 175306
Data columns (total 45 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       101884 non-null  object
 1   1       101884 non-null  object
 2   2       101884 non-null  object
 3   3       101884 non-null  object
 4   4       101884 non-null  object
 5   5       101884 non-null  object
 6   6       101884 non-null  object
 7   7       101884 non-null  object
 8   8       101884 non-null  object
 9   9       101884 non-null  object
 10  10      101884 non-null  object
 11  11      101884 non-null  object
 12  12      101884 non-null  object
 13  13      101884 non-null  object
 14  14      101884 non-null  object
 15  15      101884 non-null  object
 16  16      101884 non-null  object
 17  17      101884 non-null  object
 18  18      101884 non-null  object
 19  19      101884 non-null  object
 20  20      101884 non-null  object
 21  21      101884 non-null  object
 

In [65]:
train_data=train_data.drop([2,3,43],axis=1)
train_data=train_data.drop([0],axis=1)
test_data=test_data.drop([2,3,43],axis=1)
test_data=test_data.drop([0],axis=1)

In [66]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
'''train_data[43] = le.fit_transform(train_data[43])
test_data[43] = le.fit_transform(test_data[43])'''
train_data[4] = le.fit_transform(train_data[4])
test_data[4] = le.fit_transform(test_data[4])

In [67]:
train_data = train_data.apply(pd.to_numeric)
test_data = test_data.apply(pd.to_numeric)

In [68]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101884 entries, 1 to 175306
Data columns (total 41 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   1       101884 non-null  float64
 1   4       101884 non-null  int64  
 2   5       101884 non-null  int64  
 3   6       101884 non-null  int64  
 4   7       101884 non-null  int64  
 5   8       101884 non-null  int64  
 6   9       101884 non-null  float64
 7   10      101884 non-null  int64  
 8   11      101884 non-null  int64  
 9   12      101884 non-null  float64
 10  13      101884 non-null  float64
 11  14      101884 non-null  int64  
 12  15      101884 non-null  int64  
 13  16      101884 non-null  float64
 14  17      101884 non-null  float64
 15  18      101884 non-null  float64
 16  19      101884 non-null  float64
 17  20      101884 non-null  int64  
 18  21      101884 non-null  int64  
 19  22      101884 non-null  int64  
 20  23      101884 non-null  int64  
 21  24      10

In [69]:
'''list=[1,9,12,13,16,17,18,19,24,25,26]
for x in list:
  '''
train_data.round(decimals=2)
train_data = train_data.astype(int)
test_data.round(decimals=2)
test_data = test_data.astype(int)
train_data

,1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44
1,0,2,6,4,258,172,74,252,254,14158,8495,0,0,24,8,30,11,255,621772692,2202533631,255,0,0,0,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,0
2,0,2,14,38,734,42014,78,62,252,8395,503571,2,17,49,15,61,1387,255,1417884146,3077387971,255,0,0,0,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,0
3,1,2,8,16,364,13186,14,62,252,1572,60929,1,6,231,102,17179,11420,255,2116150707,2963114973,255,0,0,0,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,0
4,1,2,12,12,628,770,13,62,252,2740,3358,1,3,152,90,259,4991,255,1107119177,1047442890,255,0,0,0,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,0
5,0,2,10,6,534,268,33,254,252,8561,3987,2,1,47,75,2415,115,255,2436137549,1977154190,255,0,0,0,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175215,0,2,22,56,2424,68382,77,62,252,18717,543250,6,26,47,17,2347,1589,255,3903615664,3345025299,255,0,0,0,110,1221,1,109,1,1,1,1,1,1,0,0,1,21,1,0,1
175268,1,2,74,30,69997,2132,53,62,252,288572,8613,33,8,26,65,2412,90,255,556303757,246088519,255,0,0,0,946,71,0,0,1,1,1,1,1,1,0,0,0,18,1,0,1
175278,3,2,66,340,3086,426483,108,62,252,6539,914689,6,167,57,10,3358,1585,255,3046870123,1723342386,255,0,0,0,47,1254,0,0,1,1,1,1,1,1,0,0,0,2,1,0,1
175300,0,2,20,30,942,16684,49,62,252,7185,129476,7,13,52,33,3005,2479,255,5604755,575257391,255,0,0,0,47,556,0,0,1,1,1,1,1,1,0,0,0,7,1,0,1


In [70]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101884 entries, 1 to 175306
Data columns (total 41 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   1       101884 non-null  int64
 1   4       101884 non-null  int64
 2   5       101884 non-null  int64
 3   6       101884 non-null  int64
 4   7       101884 non-null  int64
 5   8       101884 non-null  int64
 6   9       101884 non-null  int64
 7   10      101884 non-null  int64
 8   11      101884 non-null  int64
 9   12      101884 non-null  int64
 10  13      101884 non-null  int64
 11  14      101884 non-null  int64
 12  15      101884 non-null  int64
 13  16      101884 non-null  int64
 14  17      101884 non-null  int64
 15  18      101884 non-null  int64
 16  19      101884 non-null  int64
 17  20      101884 non-null  int64
 18  21      101884 non-null  int64
 19  22      101884 non-null  int64
 20  23      101884 non-null  int64
 21  24      101884 non-null  int64
 22  25      101884 non-n

In [71]:
'''X=train_data.drop('attack_cat',axis=1)
Y=train_data['attack_cat']'''

"X=train_data.drop('attack_cat',axis=1)\nY=train_data['attack_cat']"

In [72]:
X = train_data.iloc[:,0:40]
Y = train_data.iloc[:,40]
C = test_data.iloc[:,40]
T = test_data.iloc[:,0:40]
print(Y)
print(X)

1         0
2         0
3         0
4         0
5         0
         ..
175215    1
175268    1
175278    1
175300    1
175306    1
Name: 44, Length: 101884, dtype: int64
        1   4   5    6      7       8    9   ...  36  37  38  39  40  41  42
1        0   2   6    4    258     172   74  ...   1   0   0   0   1   1   0
2        0   2  14   38    734   42014   78  ...   2   0   0   0   1   6   0
3        1   2   8   16    364   13186   14  ...   3   0   0   0   2   6   0
4        1   2  12   12    628     770   13  ...   3   1   1   0   2   1   0
5        0   2  10    6    534     268   33  ...  40   0   0   0   2  39   0
...     ..  ..  ..  ...    ...     ...  ...  ...  ..  ..  ..  ..  ..  ..  ..
175215   0   2  22   56   2424   68382   77  ...   1   0   0   1  21   1   0
175268   1   2  74   30  69997    2132   53  ...   1   0   0   0  18   1   0
175278   3   2  66  340   3086  426483  108  ...   1   0   0   0   2   1   0
175300   0   2  20   30    942   16684   49  ...   1   0   

In [73]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from tensorflow.keras.preprocessing import sequence
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU
from tensorflow.keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [74]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# transform data
trainX = scaler.fit_transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

In [75]:
scaler = StandardScaler()
# transform data
testT = scaler.fit_transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])

In [76]:
y_train = np.array(Y)
y_test = np.array(C)

In [77]:

X_train = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0],1, testT.shape[1]))

In [87]:


batch_size = 64

# 1. define the network
model = Sequential()
model.add(SimpleRNN(8,input_dim=40, return_sequences=True)) 
model.add(Dropout(0.1))
model.add(SimpleRNN(8, return_sequences=False)) 
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [88]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=batch_size, epochs=1)

1592/1592 [==============================] - 5s 2ms/step - loss: 0.2379 - accuracy: 0.9040


In [89]:
loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict_classes(X_test)
np.savetxt('/content/drive/MyDrive/lstm2predicted.txt',y_pred, fmt='%01d')

1635/1635 [==============================] - 3s 1ms/step - loss: 0.4319 - accuracy: 0.8349

Loss: 0.43, Accuracy: 83.49%


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [90]:
y_train1 = y_test
y_pred = model.predict_classes(X_test)
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")
print("confusion matrix")
print("----------------------------------------------")
print("accuracy")
print("%.3f" %accuracy)
print("racall")
print("%.3f" %recall)
print("precision")
print("%.3f" %precision)
print("f1score")
print("%.3f" %f1)
cm = metrics.confusion_matrix(y_train1, y_pred)
print(cm)
print("==============================================")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


confusion matrix
----------------------------------------------
accuracy
0.835
racall
0.835
precision
0.867
f1score
0.841
[[29857  7143]
 [ 1490 13815]]
